<a href="https://colab.research.google.com/github/davimbastos/Desafio_Imesao_IA_Assistente_Vagas_Emprego/blob/main/Desafio_Imersao_IA_Alura_Google.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instala a biblioteca google-generativeai (ou atualiza para a versão mais recente)
!pip install -q -U google-generativeai

In [ ]:
# Importa a biblioteca NumPy para operações numéricas.
import numpy as np

# Importa a biblioteca Pandas para manipulação de dados.
import pandas as pd

# Importa a biblioteca requests para fazer requisições HTTP.
import requests

# Importa a biblioteca json para lidar com dados JSON.
import json

# Importa a biblioteca re para usar expressões regulares.
import re

# Importa a biblioteca google-generativeai para usar a API do Google Generative AI.
import google.generativeai as genai

# Importa a função userdata do google.colab para acessar dados do usuário.
from google.colab import userdata

# Obtém a chave secreta do usuário armazenada no Colab.
my_secret_key = userdata.get('secret_key')

# Configura a API do Google Generative AI com a chave secreta do usuário.
genai.configure(api_key = my_secret_key)

In [ ]:
# Define as configurações de geração de texto.
generation_config = {
    "temperature": 1,  # Controla a aleatoriedade da saída (1 é mais aleatório).
    "top_p": 0.95,  # Limita a amostragem de tokens com base na probabilidade acumulada.
    "top_k": 0,  # Limita a amostragem aos k tokens mais prováveis.
    "max_output_tokens": 8192,  # Define o número máximo de tokens na saída gerada.
}

# Define as configurações de segurança para o conteúdo gerado.
safety_settings = [
    {
        "category": "HARM_CATEGORY_HARASSMENT",  # Categoria: Assédio
        "threshold": "BLOCK_MEDIUM_AND_ABOVE" # Bloqueia conteúdo com nível médio ou alto de assédio.
    },
    {
        "category": "HARM_CATEGORY_HATE_SPEECH", # Categoria: Discurso de ódio
        "threshold": "BLOCK_MEDIUM_AND_ABOVE" # Bloqueia conteúdo com nível médio ou alto de discurso de ódio.
    },
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", # Categoria: Conteúdo sexualmente explícito
        "threshold": "BLOCK_MEDIUM_AND_ABOVE" # Bloqueia conteúdo com nível médio ou alto de conteúdo sexualmente explícito.
    },
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT", # Categoria: Conteúdo perigoso
        "threshold": "BLOCK_MEDIUM_AND_ABOVE" # Bloqueia conteúdo com nível médio ou alto de conteúdo perigoso.
    },
]

# Cria um modelo generativo usando o modelo 'gemini-1.0-pro' com as configurações definidas.
model_content = genai.GenerativeModel(
    model_name='gemini-1.0-pro',
    generation_config=generation_config,
    safety_settings=safety_settings
)

In [ ]:
# Define a função para ler dados de uma API.
def ler_api(url):
    """
    Faz uma requisição GET para a URL especificada e retorna os dados em formato JSON.

    Args:
        url: A URL da API.

    Returns:
        Um dicionário contendo os dados da API em formato JSON, ou None se houver um erro.
    """
    try:
        # Faz a requisição GET para a URL.
        resposta = requests.get(url)

        # Verifica se a requisição foi bem-sucedida.
        resposta.raise_for_status()

        # Retorna os dados em formato JSON.
        return resposta.json()
    except requests.exceptions.RequestException as erro:
        # Imprime uma mensagem de erro se houver um problema com a requisição.
        print(f"Erro ao buscar dados da API: {erro}")

        # Retorna None se houver um erro.
        return None

# Bloco de código principal que é executado quando o script é executado diretamente.
if __name__ == "__main__":
    # Define a URL da API da Alura.
    url_api = "https://alura.com.br/api/cursos"

    # Chama a função ler_api para obter os dados da API.
    dados_cursos = ler_api(url_api)

    # Verifica se a função ler_api retornou dados.
    if dados_cursos:
        # Cria uma lista vazia para armazenar os dados dos cursos.
        tabela_cursos = []

        # Itera sobre cada curso nos dados obtidos da API.
        for curso in dados_cursos:
            # Adiciona um dicionário contendo o tempo estimado e o nome do curso à lista.
            tabela_cursos.append({
                "tempo_estimado": str(curso.get("tempo_estimado")), # Converte o tempo estimado para string.
                "nome": curso.get("nome"),  # Obtém o nome do curso.
            })

        # Cria um DataFrame do Pandas a partir da lista de cursos.
        df = pd.DataFrame(tabela_cursos)

        # Define os nomes das colunas do DataFrame.
        df.columns = ["Duracao", "Curso"]

        # Exibe o DataFrame.
        #df

In [ ]:
# Define o nome do modelo de embedding a ser usado.
model = "models/embedding-001"

# Define a função para gerar embeddings para um curso.
def embed_fn(duracao, curso):
  """
  Gera um embedding para o curso usando o modelo especificado.

  Args:
      duracao: A duração do curso.
      curso: O nome do curso.

  Returns:
      O embedding do curso como um array NumPy.
  """
  return genai.embed_content(
      model=model,  # Usa o modelo de embedding definido anteriormente.
      content=curso,  # Define o conteúdo do curso como entrada.
      title=duracao,  # Define a duração do curso como título.
      task_type="RETRIEVAL_DOCUMENT"  # Define o tipo de tarefa como recuperação de documentos.
  )["embedding"]  # Retorna o embedding do resultado.

In [ ]:
# Aplica a função embed_fn a cada linha do DataFrame para gerar embeddings.
df["Embeddings"] = df.apply(
    lambda row: embed_fn(row["Duracao"], row["Curso"]),  # Aplica a função embed_fn.
    axis=1  # Aplica a função ao longo das colunas (para cada linha).
)

# Exibe o DataFrame com a nova coluna "Embeddings".
#df

In [ ]:
# Define a função para gerar embeddings de uma consulta e buscar os cursos mais relevantes.
def gerar_e_buscar_consulta(consulta, base, model):
  """
  Gera um embedding para a consulta, calcula a similaridade com os cursos na base
  e retorna os 5 cursos mais relevantes.

  Args:
      consulta: A consulta do usuário.
      base: O DataFrame contendo os cursos e seus embeddings.
      model: O nome do modelo de embedding a ser usado.

  Returns:
      Uma lista contendo os nomes dos 5 cursos mais relevantes.
  """
  # Gera um embedding para a consulta do usuário.
  embedding_da_consulta = genai.embed_content(
      model=model,  # Usa o modelo de embedding especificado.
      content=consulta,  # Define a consulta como entrada.
      task_type="RETRIEVAL_QUERY"  # Define o tipo de tarefa como consulta de recuperação.
  )["embedding"]  # Retorna o embedding da consulta.

  # Calcula o produto escalar entre o embedding da consulta e os embeddings dos cursos.
  produtos_escalares = np.dot(
      np.stack(base["Embeddings"]),  # Empilha os embeddings dos cursos em um array.
      embedding_da_consulta  # Embedding da consulta.
  )

  # Obtém os índices dos 5 cursos com maior similaridade (produto escalar).
  indices_top_5 = np.argpartition(produtos_escalares, -5)[-5:]

  # Retorna os nomes dos 5 cursos mais relevantes.
  return base.iloc[indices_top_5]["Curso"].tolist()

In [ ]:
# Solicita ao usuário que descreva a vaga desejada.
consulta = input('Descreva a vaga que você gostaria de se candidatar:\n')

# Gera conteúdo usando o modelo de linguagem, instruindo-o a criar perguntas de entrevista.
response_model = model_content.generate_content(
    'Aja como um especialista de recrutamento. Analise a descrição da vaga e elabore cinco questões para testar os meus conhecimentos para essa vaga. Retorne as questões em um formato JSON com a seguinte estrutura: [{"pergunta": "Pergunta 1", "resposta": ""}, {"pergunta": "Pergunta 2", "resposta": ""}, {"pergunta": "Pergunta 3", "resposta": ""}, {"pergunta": "Pergunta 4", "resposta": ""}, {"pergunta": "Pergunta 5", "resposta": ""}] Descrição da vaga:' + consulta
)

# Remove caracteres extras e espaços em branco da resposta do modelo.
json_string = response_model.text.strip("```json\n").strip()

# Converte a string JSON em um objeto Python.
perguntas_json = json.loads(json_string)

# Itera sobre cada pergunta no objeto JSON.
for pergunta_dict in perguntas_json:
    # Exibe a pergunta ao usuário.
    pergunta = pergunta_dict['pergunta']
    print(pergunta)

    # Obtém a resposta do usuário.
    resposta = input("Sua resposta: ")

    # Armazena a resposta do usuário no dicionário da pergunta.
    pergunta_dict['resposta'] = resposta

# Imprime o JSON completo com as perguntas e respostas formatadas.
print(json.dumps(perguntas_json, indent=4))

In [ ]:
# Converte o objeto JSON da entrevista para uma string formatada.
entrevista = json.dumps(perguntas_json, indent=4)

# Solicita ao modelo de linguagem que analise a entrevista e gere uma avaliação do candidato.
analise_entrevista = model_content.generate_content(
    f'Analise a descrição da vaga {consulta} e avalie o desempenho do candidato com base nas perguntas e resposta contidas em {entrevista}. Defina pontos fortes e fracos do candidato em relação à vaga de emprego.'
)

# Armazena o texto da avaliação na variável "resultado".
resultado = analise_entrevista.text

In [ ]:
# Busca cursos relevantes com base na avaliação da entrevista.
recomendacao = gerar_e_buscar_consulta(resultado, df, model)

# Gera um feedback construtivo e um plano de estudo com base nos cursos recomendados.
response_model = model_content.generate_content(
    f'Escreva um feedback construtivo para o candidato e crie um plano de estudo com base nos seguintes cursos {recomendacao}'
)

# Imprime o feedback e o plano de estudo gerados pelo modelo de linguagem.
print(response_model.text)